# Lesson 2: Your First AI Agent System

**Goal:** Build a complete working AI system in 2 hours

**What you'll learn:**
- How to call local LLMs (Large Language Models via LM Studio)
- How to get structured outputs using Pydantic
- How to build AI agents that make decisions
- How to orchestrate multiple agents to work together

**Learning approach:** Start with working code, then understand by experimenting

**Prerequisites:**
This notebook assumes you have basic Python knowledge. No prior experience with AI or machine learning is required.

---

## Prerequisites Check

Make sure you have the following ready before starting:

- **LM Studio:** Running on localhost:1234  
  - LM Studio is a desktop application that runs LLMs on your computer
  - Download from: lmstudio.ai
  
- **Model loaded:** Llama 3.1 8B Instruct model  
  - This is a powerful open-source language model
  - Load it in LM Studio before running this notebook
  
- **Python packages:** httpx and pydantic installed  
  - httpx: For making HTTP requests to the LLM
  - pydantic: For data validation and structured outputs
  - Install with: `pip install httpx pydantic`

**What we're building:**
A procurement AI system that automatically processes tender opportunities:
1. Filters tenders to find relevant opportunities
2. Scores each opportunity on multiple criteria
3. Generates professional bid documents for high-scoring opportunities

In [3]:
# Test LM Studio connection
import httpx

try:
    response = httpx.get("http://localhost:1234/v1/models")
    print("LM Studio is running!")
    print(f"Model: {response.json()['data'][0]['id']}")
except Exception as e:
    print("LM Studio not running. Start it first!")
    print(f"Error: {e}")

LM Studio is running!
Model: openai/gpt-oss-20b


---

## Part 1: First LLM Call

Let's start simple: send text, get text back.

In [4]:
import asyncio
import httpx

async def simple_llm_call(prompt: str) -> str:
    """
    Make a simple call to the LLM (Large Language Model).
    
    This function sends a text prompt to the LLM and gets a text response back.
    Think of it like sending a question to ChatGPT and receiving an answer.
    """
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.post(
            "http://localhost:1234/v1/chat/completions",
            json={
                "model": "llama-3.1-8b-instruct",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7,
            }
        )
        result = response.json()
        
        # Check if the response has the expected structure
        if "choices" not in result or len(result["choices"]) == 0:
            raise ValueError(f"Unexpected API response: {result}")
        
        return result["choices"][0]["message"]["content"]

# Test it - in Jupyter notebooks we can use 'await' directly
response = await simple_llm_call("Say hello in exactly 5 words")
print(f"LLM: {response}")

LLM: Hi, I'm here to assist.


**Exercise:** Change the prompt and run again. Try:
- "Count from 1 to 10"
- "Explain AI in one sentence"
- "Write a haiku about code"

**What to observe:** Notice how the LLM responds differently to each prompt. This demonstrates the flexibility of LLMs to handle various types of text generation tasks.

---

## Part 2: Structured Outputs

**The Critical Pattern:** Make LLMs return structured data instead of free-form text

**Why this matters:**
When you ask an LLM a question, it naturally returns unstructured text. But in applications, we need:
- Predictable data formats that code can process
- Type safety and validation
- Consistent field names and structures
- The ability to use the data programmatically

**The solution:** Define a schema (data structure template) and ask the LLM to fill it in.

This is one of the most important patterns in LLM engineering. Without it, you'd need to write complex parsing logic to extract information from free-form text, which is error-prone and fragile.

In [5]:
from pydantic import BaseModel, Field
import json

# Define output structure using Pydantic
# This creates a template that the LLM will fill in
class Analysis(BaseModel):
    # Field() provides metadata: description helps the LLM understand what to put here
    is_relevant: bool = Field(description="Is this relevant?")
    
    # ge=0, le=1 means "greater than or equal to 0, less than or equal to 1"
    # This enforces that confidence must be between 0 and 1
    confidence: float = Field(description="Confidence 0-1", ge=0, le=1)
    
    reasoning: str = Field(description="Why?")

# Show the schema in JSON format
# This is what we'll send to the LLM to tell it what structure we expect
print("Schema that will be sent to the LLM:")
print(json.dumps(Analysis.model_json_schema(), indent=2))

Schema that will be sent to the LLM:
{
  "properties": {
    "is_relevant": {
      "description": "Is this relevant?",
      "title": "Is Relevant",
      "type": "boolean"
    },
    "confidence": {
      "description": "Confidence 0-1",
      "maximum": 1,
      "minimum": 0,
      "title": "Confidence",
      "type": "number"
    },
    "reasoning": {
      "description": "Why?",
      "title": "Reasoning",
      "type": "string"
    }
  },
  "required": [
    "is_relevant",
    "confidence",
    "reasoning"
  ],
  "title": "Analysis",
  "type": "object"
}


In [6]:
async def get_structured(prompt: str, schema: BaseModel) -> BaseModel:
    """
    Get LLM output that matches a specific schema.
    
    This function takes any Pydantic model as a schema and asks the LLM
    to return data matching that structure.
    
    Args:
        prompt: The question or task for the LLM
        schema: A Pydantic BaseModel class defining the expected structure
    
    Returns:
        An instance of the schema class with validated data from the LLM
    """
    
    # Convert the schema to JSON format
    schema_json = json.dumps(schema.model_json_schema(), indent=2)
    
    # Build a complete prompt that includes the original prompt plus schema instructions
    full_prompt = f"""{prompt}

Respond with valid JSON matching this schema:
{schema_json}

Rules: ONLY JSON, no markdown code blocks, no explanations"""
    
    # Call LLM with the enhanced prompt
    response = await simple_llm_call(full_prompt)
    
    # Clean the response: sometimes LLMs add markdown code blocks like ```json
    # We need to remove these to get pure JSON
    cleaned = response.strip().replace("```json", "").replace("```", "").strip()
    
    # Parse and validate the JSON against our schema
    # If validation fails, Pydantic will raise an error
    return schema.model_validate_json(cleaned)

# Test the function with our Analysis schema
result = await get_structured(
    "Analyze: 'AI Cybersecurity Platform Development'",
    Analysis
)

print("\nStructured Output:")
print(f"Relevant: {result.is_relevant}")
print(f"Confidence: {result.confidence}")
print(f"Reasoning: {result.reasoning}")


Structured Output:
Relevant: True
Confidence: 0.9
Reasoning: The subject is directly about developing an AI-based cybersecurity platform, which aligns with current interests in applying artificial intelligence to security solutions. It is a topical and relevant area for discussion, research, or project planning.


**Why this is powerful:**
- We defined a specific data structure using Pydantic (a Python library for data validation)
- The LLM filled it out according to our schema
- The data was automatically validated to ensure it matches our requirements
- We can now use this in code with proper type checking: `result.is_relevant`

**Key concept:** Instead of getting free-form text that we'd need to parse, we get structured data that's immediately usable in our application. This is one of the most important patterns in modern LLM engineering.

---

## Part 3: Complete System

Now let's build the full procurement AI system.

**System Architecture:**
```
Tender Document → Filter Agent → Rating Agent → Document Generator → Final Output
```

**How it works:**
1. **Filter Agent:** Receives a tender, determines if it's relevant to our business
2. **Rating Agent:** Scores relevant tenders on strategic fit and win probability
3. **Document Generator:** Creates professional bid documents for high-scoring opportunities

**Key design principle:**
Each agent is specialized and focused on one task. This makes the system:
- Easier to test (test each agent independently)
- Easier to debug (identify which agent has issues)
- Easier to improve (optimize one agent without affecting others)
- More maintainable (clear separation of concerns)

In [7]:
# All imports
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum
from datetime import datetime
import asyncio
import httpx
import json

In [8]:
# === DATA MODELS ===
# These define the structure of data throughout our system

# Enum: A set of predefined constant values
# This ensures categories are consistent (no typos like "cibersecurity")
class Category(str, Enum):
    CYBER = "cybersecurity"
    AI = "ai"
    SOFTWARE = "software"
    OTHER = "other"

# Input: Represents a tender document we want to process
class Tender(BaseModel):
    id: str
    title: str
    description: str
    organization: str
    estimated_value: Optional[str] = None  # Optional because not all tenders include value

# Output from Filter Agent: Decision on whether tender is relevant
class FilterResult(BaseModel):
    is_relevant: bool
    confidence: float = Field(ge=0, le=1)  # Confidence score between 0 and 1
    categories: List[Category]  # What categories apply to this tender
    reasoning: str  # Explanation of the decision

# Output from Rating Agent: Detailed scoring of a tender
class RatingResult(BaseModel):
    overall_score: float = Field(ge=0, le=10)  # Overall opportunity score out of 10
    strategic_fit: float = Field(ge=0, le=10)  # How well it aligns with our strategy
    win_probability: float = Field(ge=0, le=10)  # Likelihood we could win this bid
    strengths: List[str]  # List of our advantages for this opportunity
    risks: List[str]  # List of potential challenges or concerns
    recommendation: str  # Final recommendation text

# Output from Document Generator: A structured bid document
class BidDoc(BaseModel):
    executive_summary: str  # High-level overview for decision makers
    technical_approach: str  # How we would execute the project
    value_proposition: str  # Why the client should choose us

print("Data models defined successfully")

Data models defined successfully


In [16]:
# === LLM SERVICE ===
# This class encapsulates all communication with the LLM

class LLM:
    """
    A service for interacting with the LLM API.
    
    This wraps the API calls in a convenient interface and handles
    structured output generation consistently across all agents.
    """
    def __init__(self):
        self.url = "http://localhost:1234/v1"
        self.model = "llama-3.1-8b-instruct"
    
    def _clean_json(self, text: str) -> str:
        """
        Remove markdown artifacts and extract valid JSON from LLM response.
        
        This method handles various formats that LLMs might return:
        - Markdown code blocks (```json ... ```)
        - Extra text before or after the JSON
        - Model-specific tokens
        
        It uses brace counting to find the actual JSON object boundaries.
        """
        cleaned = text.strip()
        
        # Remove markdown code blocks
        if cleaned.startswith("```json"):
            cleaned = cleaned[7:]
        elif cleaned.startswith("```"):
            cleaned = cleaned[3:]
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3]
        
        cleaned = cleaned.strip()
        
        # Find the JSON object by looking for balanced braces
        # This is more robust than regex because it handles nested objects
        start_idx = cleaned.find('{')
        if start_idx == -1:
            return cleaned
        
        # Count braces to find the matching closing brace
        brace_count = 0
        end_idx = -1
        
        for i, char in enumerate(cleaned[start_idx:], start_idx):
            if char == '{':
                brace_count += 1
            elif char == '}':
                brace_count -= 1
                if brace_count == 0:
                    end_idx = i
                    break
        
        if end_idx != -1:
            return cleaned[start_idx:end_idx + 1]
        
        return cleaned
    
    async def generate(self, prompt: str, schema: BaseModel, 
                       system: str = "", temp: float = 0.1) -> BaseModel:
        """
        Generate structured output from the LLM.
        
        Args:
            prompt: The main instruction or question
            schema: Pydantic model defining the expected output structure
            system: System message to set the LLM's role/behavior (optional)
            temp: Temperature parameter (0.0 = focused, 1.0 = creative)
        
        Returns:
            Validated instance of the schema with LLM-generated data
        """
        messages = []
        
        # System message sets the context and role for the LLM
        if system:
            messages.append({"role": "system", "content": system})
        
        # Add schema to the prompt so LLM knows what format to use
        schema_json = json.dumps(schema.model_json_schema(), indent=2)
        full_prompt = f"""{prompt}

Respond ONLY with valid JSON matching:
{schema_json}

IMPORTANT: Return ONLY the JSON object, nothing else. No explanations, no markdown, no extra text.
Start with {{ and end with }}"""
        
        messages.append({"role": "user", "content": full_prompt})
        
        # Make the API call with extended timeout for longer generations
        async with httpx.AsyncClient(timeout=120) as client:
            r = await client.post(
                f"{self.url}/chat/completions",
                json={
                    "model": self.model,
                    "messages": messages,
                    "temperature": temp,
                    "max_tokens": 2000,  # Maximum length of response
                }
            )
            
            # Check for API errors
            if r.status_code != 200:
                raise ValueError(f"API error: {r.status_code} - {r.text}")
            
            result = r.json()
            if "choices" not in result or len(result["choices"]) == 0:
                raise ValueError(f"Unexpected API response: {result}")
            
            content = result["choices"][0]["message"]["content"]
        
        # Clean the JSON using the same method as in src/procurement_ai/services/llm.py
        clean_json = self._clean_json(content)
        
        # Validate that we have something that looks like JSON
        if not clean_json.startswith('{') or not clean_json.endswith('}'):
            print(f"\nWarning: Response doesn't look like JSON")
            print(f"Raw content (first 300 chars): {content[:300]}")
            print(f"Cleaned (first 300 chars): {clean_json[:300]}")
            raise ValueError(f"Response doesn't look like valid JSON")
        
        # Parse and validate against schema
        try:
            # First parse as JSON to catch JSON errors
            parsed = json.loads(clean_json)
            # Then validate against the Pydantic model
            return schema.model_validate(parsed)
        except json.JSONDecodeError as e:
            print(f"\nJSON parsing error: {e}")
            print(f"Raw content (first 300 chars): {content[:300]}")
            print(f"Cleaned JSON (first 300 chars): {clean_json[:300]}")
            raise
        except Exception as e:
            print(f"\nValidation error: {e}")
            print(f"Cleaned JSON (first 300 chars): {clean_json[:300]}")
            raise

# Create a single instance to reuse across all agents
llm = LLM()
print("LLM service initialized and ready")

LLM service initialized and ready


### Agent 1: Filter Agent

**Purpose:** Determines if a tender is relevant to our business

**How it works:**
1. Receives a Tender object
2. Analyzes the title and description
3. Returns a FilterResult with relevance decision, confidence score, and categories

In [17]:
class FilterAgent:
    """
    Filters tenders to identify relevant opportunities.
    
    This agent acts as a first-pass filter to avoid wasting time
    analyzing tenders that don't match our business focus.
    """
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def filter(self, tender: Tender) -> FilterResult:
        """
        Analyze a tender and determine if it's relevant.
        
        Args:
            tender: The tender document to analyze
        
        Returns:
            FilterResult with relevance decision and reasoning
        """
        # Craft a clear prompt explaining the task
        prompt = f"""Analyze this tender:

TITLE: {tender.title}
DESCRIPTION: {tender.description}

RELEVANCE CRITERIA:
- RELEVANT if: cybersecurity, AI/ML, or software development
- NOT relevant if: hardware, construction, non-tech services

Determine if this is relevant and assign appropriate categories."""
        
        # Use low temperature (0.1) for consistent, deterministic filtering
        return await self.llm.generate(
            prompt, FilterResult,
            system="You are a procurement analyst. Be precise and consistent in your judgments.",
            temp=0.1
        )

print("Filter agent defined successfully")

Filter agent defined successfully


### Agent 2: Rating Agent

**Purpose:** Scores opportunities on strategic fit and win probability

**How it works:**
1. Receives a Tender and its categories from the Filter Agent
2. Evaluates on multiple dimensions (strategic fit, win probability)
3. Returns RatingResult with scores, strengths, risks, and recommendation

In [18]:
class RatingAgent:
    """
    Rates opportunities on multiple criteria.
    
    This agent performs deeper analysis on relevant tenders to help
    prioritize which opportunities to pursue.
    """
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def rate(self, tender: Tender, cats: List[str]) -> RatingResult:
        """
        Rate a tender opportunity.
        
        Args:
            tender: The tender to rate
            cats: Categories assigned by the Filter Agent
        
        Returns:
            RatingResult with detailed scoring and analysis
        """
        prompt = f"""Rate this opportunity:

TENDER: {tender.title}
ORGANIZATION: {tender.organization}
ESTIMATED VALUE: {tender.estimated_value}
CATEGORIES: {', '.join(cats)}

Provide scores (0-10 scale):
- Strategic fit: How well does this align with our capabilities?
- Win probability: What are our chances of winning this bid?
- Overall score: Combined assessment

Also identify:
- Top 3 strengths (why we'd be good fit)
- Top 3 risks (potential challenges)
- Clear recommendation (pursue, consider, or skip)"""
        
        # Use low temperature for consistent scoring
        return await self.llm.generate(
            prompt, RatingResult,
            system="You are a business analyst. Be realistic and data-driven in your assessments.",
            temp=0.1
        )

print("Rating agent defined successfully")

Rating agent defined successfully


### Agent 3: Document Generator

**Purpose:** Creates professional bid documents

**How it works:**
1. Receives tender information, categories, and strengths from previous agents
2. Generates three key sections: executive summary, technical approach, and value proposition
3. Uses higher temperature for more creative, varied writing

In [19]:
class DocGenerator:
    """
    Generates professional bid documents.
    
    This agent takes the analysis from previous agents and creates
    polished, professional content for bid submissions.
    """
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def generate(self, tender: Tender, cats: List[str], 
                       strengths: List[str]) -> BidDoc:
        """
        Generate a bid document.
        
        Args:
            tender: The tender we're bidding on
            cats: Relevant categories
            strengths: Our key strengths (from Rating Agent)
        
        Returns:
            BidDoc with executive summary, technical approach, and value proposition
        """
        prompt = f"""Create a professional bid document for:

TENDER: {tender.title}
CLIENT: {tender.organization}
OUR EXPERTISE: {', '.join(cats)}
KEY STRENGTHS: {', '.join(strengths)}

Generate three sections:

1. Executive Summary (2-3 paragraphs)
   - High-level overview of our proposed solution
   - Why we're the right choice
   - Expected outcomes

2. Technical Approach (2-3 paragraphs)
   - How we would execute the project
   - Methodology and tools
   - Timeline and milestones

3. Value Proposition (2 paragraphs)
   - Unique benefits we bring
   - ROI and long-term value

Tone: Professional, confident, client-focused
Style: Specific and concrete, not generic marketing language"""
        
        # Use higher temperature (0.7) for more creative, varied writing
        return await self.llm.generate(
            prompt, BidDoc,
            system="You are an expert proposal writer with 10+ years of experience winning government contracts.",
            temp=0.7  # Higher temperature for creativity
        )

print("Document generator defined successfully")

Document generator defined successfully


### Orchestrator: Coordinating the Agents

**Purpose:** Manages the workflow and coordinates all three agents

**How it works:**
1. Receives a tender as input
2. Runs the Filter Agent first
3. If relevant, runs the Rating Agent
4. If high-scoring, runs the Document Generator
5. Returns the combined results

In [20]:
class Orchestrator:
    """
    Orchestrates the multi-agent workflow.
    
    This class coordinates the three agents, managing data flow between them
    and implementing business logic (e.g., skip low-scoring tenders).
    """
    def __init__(self):
        self.llm = LLM()
        self.filter = FilterAgent(self.llm)
        self.rater = RatingAgent(self.llm)
        self.doc_gen = DocGenerator(self.llm)
    
    async def process(self, tender: Tender):
        """
        Process a tender through the complete workflow.
        
        Args:
            tender: The tender to process
        
        Returns:
            Dictionary with filter results, rating, and document (if generated)
            Returns None if tender is filtered out as not relevant
        """
        print(f"\n{'='*60}")
        print(f"Processing: {tender.title[:50]}...")
        print(f"{'='*60}")
        
        start = datetime.now()
        
        # Step 1: Filter - Determine relevance
        print("\n[1/3] Filtering...")
        filter_result = await self.filter.filter(tender)
        print(f"  Relevant: {filter_result.is_relevant} (Confidence: {filter_result.confidence:.0%})")
        print(f"  Categories: {[c.value for c in filter_result.categories]}")
        
        # Business logic: Skip if not relevant or low confidence
        if not filter_result.is_relevant or filter_result.confidence < 0.6:
            print("\n  Decision: Skipping (not relevant or low confidence)")
            return None
        
        # Step 2: Rate - Score the opportunity
        print("\n[2/3] Rating...")
        cats = [c.value for c in filter_result.categories]
        rating = await self.rater.rate(tender, cats)
        print(f"  Overall Score: {rating.overall_score:.1f}/10")
        print(f"  Strategic Fit: {rating.strategic_fit:.1f}/10")
        print(f"  Win Probability: {rating.win_probability:.1f}/10")
        
        # Business logic: Only generate documents for high-scoring opportunities
        if rating.overall_score < 7.0:
            print("\n  Decision: Skipping document generation (score below 7.0 threshold)")
            return {"filter": filter_result, "rating": rating}
        
        # Step 3: Generate - Create bid document
        print("\n[3/3] Generating document...")
        doc = await self.doc_gen.generate(tender, cats, rating.strengths)
        print("  Status: Document ready")
        
        elapsed = (datetime.now() - start).total_seconds()
        print(f"\n  Total processing time: {elapsed:.1f} seconds")
        
        return {
            "filter": filter_result,
            "rating": rating,
            "document": doc,
            "time": elapsed
        }

# Create orchestrator instance
orchestrator = Orchestrator()
print("Orchestrator initialized and ready to process tenders")

Orchestrator initialized and ready to process tenders


---

## Test Data

Let's create some sample tenders to test our system. We have three different types:
1. A highly relevant AI/Cybersecurity project
2. A completely irrelevant furniture procurement
3. A moderately relevant healthcare software project

In [14]:
# Sample tenders covering different scenarios
tenders = [
    # Tender 1: High relevance - matches our expertise perfectly
    Tender(
        id="T001",
        title="AI-Powered Cybersecurity Platform",
        description="""Develop AI-based threat detection system for government 
        infrastructure. Must use machine learning for anomaly detection and 
        integrate with existing SIEM tools. Real-time monitoring required.""",
        organization="National Cyber Agency",
        estimated_value="3.2M EUR"
    ),
    
    # Tender 2: Not relevant - completely outside our domain
    Tender(
        id="T002",
        title="Office Furniture Supply",
        description="""Supply ergonomic office furniture for 500 workstations 
        including desks, chairs, and storage. Must meet ISO 9001 standards
        and provide 5-year warranty.""",
        organization="Ministry of Public Works",
        estimated_value="450K EUR"
    ),
    
    # Tender 3: Moderately relevant - software but different domain
    Tender(
        id="T003",
        title="Custom Healthcare CRM System",
        description="""Build cloud-based CRM for healthcare network. Must 
        handle patient data, appointments, billing. GDPR compliant with 
        mobile app for doctors and patients.""",
        organization="Regional Health Authority",
        estimated_value="1.8M EUR"
    ),
]

print(f"Test data prepared: {len(tenders)} tenders ready for processing")

Test data prepared: 3 tenders ready for processing


---

## Run the System

Now let's process all three test tenders through our system.
Each tender will go through the filter, rating, and potentially document generation steps.

In [15]:
# Process all tenders sequentially
# Store results for later analysis
results = []

for tender in tenders:
    result = await orchestrator.process(tender)
    results.append({"tender": tender, "result": result})

print("\n\nAll tenders processed!")


Processing: AI-Powered Cybersecurity Platform...

[1/3] Filtering...


ValidationError: 1 validation error for FilterResult
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='<|channel|>final <|const...rvices are requested."}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid

---

## Results Analysis

Let's analyze the aggregate results across all processed tenders.

In [ ]:
print("\n\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60 + "\n")

# Count tenders at each stage
relevant = sum(1 for r in results if r["result"] and "filter" in r["result"])
high_scored = sum(1 for r in results if r["result"] and "rating" in r["result"] 
                  and r["result"]["rating"].overall_score >= 7.0)
docs_made = sum(1 for r in results if r["result"] and "document" in r["result"])

print(f"Total Tenders Processed: {len(tenders)}")
print(f"Relevant (passed filter): {relevant}")
print(f"High Scored (>= 7.0): {high_scored}")
print(f"Documents Generated: {docs_made}")

# Calculate processing time
total_time = sum(r["result"]["time"] for r in results if r["result"] and "time" in r["result"])
print(f"\nTotal Processing Time: {total_time:.1f} seconds")
if len(tenders) > 0:
    print(f"Average per Tender: {total_time/len(tenders):.1f} seconds")

print("\n" + "="*60)

### Detailed Results

Now let's look at each tender in detail to see how it was processed.

In [ ]:
for i, item in enumerate(results, 1):
    tender = item["tender"]
    result = item["result"]
    
    print(f"\n{'─'*60}")
    print(f"TENDER {i}: {tender.title}")
    print(f"{'─'*60}")
    print(f"Organization: {tender.organization}")
    print(f"Value: {tender.estimated_value}")
    
    # Case 1: Filtered out completely
    if not result:
        print("\nStatus: FILTERED OUT (not relevant)")
        continue
    
    # Case 2: Has filter results
    if "filter" in result:
        f = result["filter"]
        print(f"\nFilter Results:")
        print(f"  Relevant: {f.is_relevant}")
        print(f"  Confidence: {f.confidence:.0%}")
        print(f"  Categories: {[c.value for c in f.categories]}")
        print(f"  Reasoning: {f.reasoning}")
    
    # Case 3: Has rating results
    if "rating" in result:
        r = result["rating"]
        print(f"\nRating Results:")
        print(f"  Overall Score: {r.overall_score:.1f}/10")
        print(f"  Strategic Fit: {r.strategic_fit:.1f}/10")
        print(f"  Win Probability: {r.win_probability:.1f}/10")
        print(f"\n  Strengths:")
        for s in r.strengths:
            print(f"    - {s}")
        print(f"\n  Risks:")
        for risk in r.risks:
            print(f"    - {risk}")
        print(f"\n  Recommendation: {r.recommendation[:150]}...")
    
    # Case 4: Document was generated
    if "document" in result:
        print(f"\nStatus: DOCUMENT GENERATED")
        print(f"\nExecutive Summary (excerpt):")
        print(f"  {result['document'].executive_summary[:250]}...")
        print(f"\nProcessing Time: {result['time']:.1f} seconds")
    elif "rating" in result:
        print(f"\nStatus: RATED BUT NO DOCUMENT (score below threshold)")
    
print("\n" + "="*60)

---

## What You Just Built

Congratulations! You just built a complete AI agent system that:

1. **Filters** tenders by relevance (classification task)
2. **Rates** opportunities on multiple dimensions (scoring task)
3. **Generates** professional documents (content creation task)
4. **Orchestrates** multiple agents in a workflow (coordination task)

**Key Patterns You Learned:**

**1. LLM API calls:** Making HTTP requests to interact with a Language Model
- Similar to how you'd call any web API
- Send a request with your prompt, receive a response with generated text

**2. Structured outputs:** Using Pydantic schemas to get predictable, validated data
- Instead of parsing free-form text, we get data structures we can use directly
- The LLM fills in fields according to our defined schema
- Automatic validation ensures data integrity

**3. Prompt engineering:** Writing clear instructions to guide LLM behavior
- Specific prompts lead to better results
- Including examples, constraints, and output format requirements helps
- Iterative refinement improves accuracy

**4. Temperature control:** Balancing precision versus creativity
- Low temperature (0.1-0.3): Focused, deterministic, good for classification
- High temperature (0.7-1.0): Creative, varied, good for content generation
- This parameter controls the randomness in LLM outputs

**5. Multi-agent design:** Breaking complex tasks into specialized components
- Each agent has a specific role and expertise
- Sequential workflow: Output of one agent feeds into the next
- Easier to test, debug, and improve individual components

---

## Experiments to Try

**1. Temperature Impact:**
- Change temperature in FilterAgent to 0.5
- Run again, compare confidence scores
- Question to explore: Does it change decisions? Why might that happen?

**2. Prompt Engineering:**
- Modify FilterAgent prompt to be more specific
- Add examples of relevant tenders in the prompt
- Question: Does accuracy improve? What happens if you make it too specific?

**3. Add Your Own Tender:**
- Create a new Tender object with real or fictional data
- Process it through the system
- Analyze: Did it get categorized correctly? What was the score?

**4. Scoring Threshold:**
- Change the 7.0 threshold in the orchestrator
- Try 5.0 (more lenient) or 8.0 (more strict)
- Observe: How does it affect which documents get generated?

**5. Error Handling:**
- What happens if LM Studio is not running?
- Try to handle errors gracefully with try/except blocks
- Add timeout handling for slow responses

---

## Key Concepts Explained

**What is an Agent?**
An agent is a component that uses an LLM to perform a specific task. It has:
- A clear responsibility (filtering, rating, or generating)
- A specialized prompt tailored to its task
- Input and output data structures
- Configuration parameters (like temperature)

**What is Orchestration?**
Orchestration is the coordination of multiple agents to complete a complex workflow. The orchestrator:
- Determines the order of operations
- Passes data between agents
- Handles conditional logic (e.g., skip document generation if score is low)
- Manages errors and provides feedback

**Why Use Local LLMs?**
Running LLMs locally (like with LM Studio) offers:
- Privacy: Your data never leaves your machine
- Cost: No per-token API fees
- Speed: No network latency to external services
- Control: You can choose and customize the model

---

## Next Steps

**Notebook 02:** Experiments with prompt engineering  
**Notebook 03:** Real data collection and validation  
**Notebook 04:** Production code structure and best practices  

**Advanced topics to explore:**
- Parallel processing of multiple tenders
- Caching LLM responses to avoid redundant calls
- Adding a feedback loop to improve agent performance
- Implementing retrieval-augmented generation (RAG) for domain knowledge
- Monitoring and logging for production deployments

---

## Save Your Work

```bash
# In terminal:
git add learn/02_mvp_complete.ipynb
git commit -m "Complete: First working AI agent system

Built 3-agent orchestration:
- Filter: Classifies tenders by relevance with confidence scoring
- Rating: Multi-dimensional scoring of opportunities
- Generator: Professional document creation with variable creativity

Processing time: Approximately 8s per tender on M1 Mac
Cost: $0 (local LLM via LM Studio)

Ready for experimentation phase."

git tag -a v0.1-mvp -m "First working MVP complete"
```

**What this commit represents:**
- A working baseline you can return to
- Clear documentation of what was accomplished
- A reference point for measuring improvements